In [1]:
import pandas as pd

In [2]:
def drop_niet_blokken(df):
    '''Unit is een combinatie van 4 cijfers als het om een overkoepelende term gaat.
    Door alleen de regels over te houden waar unit uit meer dan 4 characters bestaat,
    houden we de regels over blokken over.'''
    dataf = df.copy()
    return dataf.loc[dataf.Unit.str.len() > 4, ]

In [3]:

def get_vaste_staf_namelist(excelfile):
    namen = pd.read_excel(excelfile, sheet_name='Sheet1')

    return list(namen['Volledige naam'].values)

VASTE_STAF_NAMEN = get_vaste_staf_namelist('../Data/HSR Vaste staf 01-05-2022.XLSX')

def check_if_vaste_staf(naam, vaste_staf_namen=VASTE_STAF_NAMEN):

    return naam in vaste_staf_namen

def check_if_vaste_staf_achternaam(achternaam, vaste_staf_namen=VASTE_STAF_NAMEN):
    vaste_staf_achternamen = [naam.split(' ')[-1].lower() for naam in vaste_staf_namen]

    return achternaam.lower() in vaste_staf_achternamen

In [4]:
check_if_vaste_staf_achternaam('Ruwa')

False

In [5]:
naam_mapping = pd.read_excel('../Data/HSR Vaste staf 01-05-2022.XLSX', sheet_name='NaamMapping')

def onderwijsnaam_naar_onderzoeksnaam(naam, naam_mapping=naam_mapping):

    return naam_mapping.loc[naam_mapping.OnderwijsNaam == naam, 'Volledige naam'].values[0]


In [6]:
onderwijsnaam_naar_onderzoeksnaam('SMG Zwakhalen')

'Sandra Zwakhalen'

In [8]:
files = ['../Data/BROS HSR realisations 2019-2020, 14.5.2022-Blokcoordinatoren en planningsgroepsleden.xlsx',
        '../Data/BROS HSR realisations 2020-2021, 11.2.2022-Blokcoordinatoren en planningsgroepsleden.xlsx',
        '../Data/BROS HSR realisations 2021-2022, 14.5.2022-Blokcoordinatoren en planningsgroepsleden.xlsx']

onderwijs = pd.DataFrame()

# zet alle jaren in 1 df
for jaar, filename in zip(range(2019, 2022), files):
    onderwijs = pd.concat([onderwijs, (pd.read_excel(filename,
                                                     sheet_name='Realisations')
                                       .dropna(subset=['Unit'])
                                       .assign(Year=jaar)
                                       .pipe(drop_niet_blokken)
                                       .drop_duplicates())],
                          axis=0)
    
onderwijs['UnitYear'] = [f"{unit}_{year}" for unit, year in zip(onderwijs['Unit'], onderwijs['Year'])]
onderwijs['Prefix'] = onderwijs['Prefix'].fillna('')
onderwijs['Naam'] = (onderwijs.Initials + " " + onderwijs.Prefix + " " + onderwijs.Name).str.replace("  ", " ").str.replace("  ", " ")

# selecteer alleen vaste staf
onderwijs['is_vaste_staf'] = onderwijs.Name.apply(check_if_vaste_staf_achternaam)
onderwijs = onderwijs.loc[onderwijs.is_vaste_staf, ]

# zorg dat namen gelijk geschreven zijn als in Pure/Onderzoek
missing_before = onderwijs['Naam'].isnull().sum()
onderwijs['Naam'] = [onderwijsnaam_naar_onderzoeksnaam(naam) for naam in onderwijs['Naam'].values]
missing_after = onderwijs['Naam'].isnull().sum()
assert missing_before == missing_after

onderwijs.to_csv('../Data/onderwijs.csv', index=False)

onderwijs

,Unit,Unit name,Category,Name,Prefix,Initials,Department,Role name,Year,UnitYear,Naam,is_vaste_staf
3,GZW1021,Een Leven Lang Gezond,B,Zwakhalen,,SMG,HSR,Member Planning Group,2019,GZW1021_2019,Sandra Zwakhalen,True
4,GZW1022,Bedreigingen van Gezondheid,B,Haastregt,van,JCM,HSR,Member Planning Group,2019,GZW1022_2019,Jolanda van Haastregt,True
5,GZW1024,"Gezondheid, Voeding en Bewegen",C,Zijlstra,,GAR,HSR,Coordinator Planning Group,2019,GZW1024_2019,G.A. Rixt Zijlstra,True
6,GZW1025,Zorg(en) voor Gezondheid,C,Groothuis,,S,HSR,Coordinator Planning Group,2019,GZW1025_2019,Siebren Groothuis,True
7,GZW1025,Zorg(en) voor Gezondheid,B,Evers,,SMAA,HSR,Member Planning Group,2019,GZW1025_2019,Silvia Evers,True
...,...,...,...,...,...,...,...,...,...,...,...,...
159,HDTCOOR,Master Health & Digital Technology,C,Spreeuwenberg,,MD,HSR,Programme Coordinator,2021,HDTCOOR_2021,Marieke Spreeuwenberg,True
162,SSC2053,Public health policymaking,C,Ruwaard,,D,HSR,Coordinator Planning Group,2021,SSC2053_2021,Dirk Ruwaard,True
163,SSC2053,Public health policymaking,B,Pavlova,,MI,HSR,Member Planning Group,2021,SSC2053_2021,Milena Pavlova,True
164,SSC2053,Public health policymaking,B,Drost,,RMWA,HSR,Member Planning Group,2021,SSC2053_2021,Ruben Drost,True


In [10]:
onderwijs.loc[~onderwijs.Unit.str.contains('COOR', na=False),]

,Unit,Unit name,Category,Name,Prefix,Initials,Department,Role name,Year,UnitYear,Naam,is_vaste_staf
3,GZW1021,Een Leven Lang Gezond,B,Zwakhalen,,SMG,HSR,Member Planning Group,2019,GZW1021_2019,Sandra Zwakhalen,True
4,GZW1022,Bedreigingen van Gezondheid,B,Haastregt,van,JCM,HSR,Member Planning Group,2019,GZW1022_2019,Jolanda van Haastregt,True
5,GZW1024,"Gezondheid, Voeding en Bewegen",C,Zijlstra,,GAR,HSR,Coordinator Planning Group,2019,GZW1024_2019,G.A. Rixt Zijlstra,True
6,GZW1025,Zorg(en) voor Gezondheid,C,Groothuis,,S,HSR,Coordinator Planning Group,2019,GZW1025_2019,Siebren Groothuis,True
7,GZW1025,Zorg(en) voor Gezondheid,B,Evers,,SMAA,HSR,Member Planning Group,2019,GZW1025_2019,Silvia Evers,True
...,...,...,...,...,...,...,...,...,...,...,...,...
152,HMS4802,The Enterpreneurial Healthcare Professional,B,Govers,,MJG,HSR,Member Planning Group,2021,HMS4802_2021,Mark Govers,True
162,SSC2053,Public health policymaking,C,Ruwaard,,D,HSR,Coordinator Planning Group,2021,SSC2053_2021,Dirk Ruwaard,True
163,SSC2053,Public health policymaking,B,Pavlova,,MI,HSR,Member Planning Group,2021,SSC2053_2021,Milena Pavlova,True
164,SSC2053,Public health policymaking,B,Drost,,RMWA,HSR,Member Planning Group,2021,SSC2053_2021,Ruben Drost,True


In [9]:
from itertools import combinations
from collections import Counter

all_tuples = []
for unityear in onderwijs.UnitYear.unique():
    
    # get rows of that unityear
    # get staff in that unityear
    staff = onderwijs.loc[onderwijs.UnitYear == unityear, 'Naam'].values

    # get all combinations of 2 of staff in that unityear
    for c in list(combinations(staff, 2)):
        # print(unityear, c)
        all_tuples.append(c)
        
Counter(all_tuples)
        

Counter({('Siebren Groothuis', 'Silvia Evers'): 2,
         ('Siebren Groothuis', 'Wim Groot'): 2,
         ('Silvia Evers', 'Wim Groot'): 4,
         ('Arianne Elissen', 'G.A. Rixt Zijlstra'): 1,
         ('Nynke de Jong', 'Bram de Boer'): 4,
         ('Nynke de Jong', 'Siebren Groothuis'): 3,
         ('Nynke de Jong', 'Jan Hamers'): 3,
         ('Bram de Boer', 'Siebren Groothuis'): 3,
         ('Bram de Boer', 'Jan Hamers'): 3,
         ('Siebren Groothuis', 'Jan Hamers'): 3,
         ('Agnes Paulus', 'Ruben Drost'): 3,
         ('Agnes Paulus', 'Arianne Elissen'): 3,
         ('Agnes Paulus', 'Silvia Evers'): 3,
         ('Agnes Paulus', 'Mickaël Hiligsmann'): 3,
         ('Ruben Drost', 'Arianne Elissen'): 3,
         ('Ruben Drost', 'Silvia Evers'): 4,
         ('Ruben Drost', 'Mickaël Hiligsmann'): 6,
         ('Arianne Elissen', 'Silvia Evers'): 3,
         ('Arianne Elissen', 'Mickaël Hiligsmann'): 3,
         ('Silvia Evers', 'Mickaël Hiligsmann'): 4,
         ('Jolanda van 

In [10]:
onderwijs[['Name', 'Naam']].groupby('Naam').first().sort_values('Name').to_csv('../Data/onderwijsnamen.csv')